In [1]:
#!git clone https://github.com/usnistgov/trec_eval.git && cd trec_eval && make

In [2]:
import os
import json
import numpy as np
import pandas as pd
from typing import Dict
from tqdm import tqdm
from re import compile
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
#import unicodedata
from contractions import fix as fix_contractions
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from functools import partial

/home/raul/Escritorio/extra/misis/rl/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Funciones

In [3]:
# Cargar un modelo preentrenado para embeddings semánticos, utilizado en consultas semánticas
sentence_transformer_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Conjunto de stopwords en inglés y un stemmer basado en la lengua inglesa
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer(language='english')

# Expresiones regulares precompiladas para limpieza de texto
pattern_newline = compile(r'[\n\t\u200e]')  # Elimina saltos de línea, tabulaciones y caracteres no deseados
pattern_non_alphanumeric = compile(r'[^a-z0-9]')  # Elimina caracteres no alfanuméricos
pattern_multiple_spaces = compile(r' +')  # Elimina múltiples espacios consecutivos

def clean_text(text: str) -> str:
    """
    Limpia y normaliza el texto, expandiendo contracciones, eliminando caracteres no deseados,
    y aplicando stemming y eliminación de stopwords.

    Args:
        text (str): El texto de entrada a ser limpiado.

    Returns:
        str: Texto limpio y normalizado.
    """    
    # Expande las contracciones para evitar inconsistencias (ej. "don't" -> "do not")
    cln_text = fix_contractions(text)
    
    # Convierte el texto a minúsculas
    cln_text = cln_text.lower()
    
    # Normalización Unicode (comentado por el momento)
    #cln_text = normalize('NFKD', cln_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Elimina saltos de línea y otros caracteres no deseados
    cln_text = pattern_newline.sub(' ', cln_text)
    
    # Elimina todos los caracteres no alfanuméricos (ej. puntuación)
    cln_text = pattern_non_alphanumeric.sub(' ', cln_text)
    
    # Tokeniza el texto y aplica stemming a cada palabra, eliminando las stopwords
    tokens = [stemmer.stem(word) for word in word_tokenize(cln_text) if word not in stop_words]
    
    # Junta las palabras en una sola cadena de texto
    cln_text = ' '.join(tokens)
    
    # Elimina múltiples espacios consecutivos y recorta espacios en los extremos
    cln_text = pattern_multiple_spaces.sub(' ', cln_text).strip()
    
    return cln_text

def tokenizer(text:str)-> list:
    """
    Tokeniza el texto en unigramas y bigramas para enriquecer la representación del texto.
    
    Args:
        text (str): Texto de entrada.

    Returns:
        list: Lista de unigramas y bigramas.
    """    
    tokens = text.split()
    
    # Crear unigramas
    unigrams = tokens
    
    # Crear bigramas (combinaciones de palabras consecutivas)
    bigrams = [f"{tokens[i]} {tokens[i + 1]}" for i in range(len(tokens) - 1)]
    
    # Retorna la lista de unigramas y bigramas juntos
    return unigrams + bigrams

def sintactic_query_bm5(query: str, bm5_instance: BM25Okapi) -> np.array:
    """
    Realiza una consulta sintáctica utilizando el algoritmo BM25 para recuperar documentos relevantes.

    Args:
        query (str): Consulta de entrada.
        bm5_instance (BM25Okapi): Instancia de BM25 previamente inicializada con el corpus.

    Returns:
        np.array: Array de puntajes de BM25 para cada documento en el corpus.
    """    
    
    # Limpia y tokeniza la consulta usando el tokenizer definido (unigramas y bigramas)
    tokenized_query = tokenizer(clean_text(query))
    
    # Obtiene los puntajes sintácticos usando el modelo BM25
    scores = bm5_instance.get_scores(tokenized_query)
    
    return scores

def sintactic_query_tfidf(query: str, corpus_tfidf_matrix: np.array, 
                          tfidf_vectorizer: TfidfVectorizer) -> np.array:
    """
    Realiza una consulta sintáctica usando el modelo TF-IDF y calcula la similitud del coseno
    entre la consulta y el corpus.

    Args:
        query (str): Consulta de entrada.
        corpus_tfidf_matrix (np.array): Matriz TF-IDF del corpus.
        tfidf_vectorizer (TfidfVectorizer): Vectorizador TF-IDF previamente entrenado.

    Returns:
        np.array: Array de puntajes de similitud del coseno para cada documento.
    """    
    
    # Limpia la consulta (sin stemming ni stopwords, porque el modelo TF-IDF se entrenó con el texto original)
    cln_query = pattern_newline.sub(' ', query)
    cln_query = pattern_multiple_spaces.sub(' ', cln_query).strip()
    
    # Transforma la consulta en un vector TF-IDF
    query_tfidf = tfidf_vectorizer.transform([cln_query])
    
    # Calcula la similitud del coseno entre la consulta y cada documento en el corpus
    scores = (query_tfidf @ corpus_tfidf_matrix.T).toarray()[0]
    
    return scores

def semantic_query(query: str, corpus_embeddings_matrix: np.array, 
                   sentence_transformer_model: SentenceTransformer) -> np.array:
    """
    Realiza una consulta semántica utilizando embeddings preentrenados con un modelo de 
    transformer y calcula la similitud entre los embeddings de la consulta y el corpus.

    Args:
        query (str): Consulta de entrada.
        corpus_embeddings_matrix (np.array): Matriz de embeddings del corpus.
        sentence_transformer_model (SentenceTransformer): Modelo preentrenado para generar embeddings.

    Returns:
        np.array: Array de puntajes de similitud del coseno para cada documento.
    """    
    
    # Limpia la consulta (sin stemming ni eliminación de stopwords para mantener el contexto)
    cln_query = pattern_newline.sub(' ', query)
    cln_query = pattern_multiple_spaces.sub(' ', cln_query).strip()
    
    # Genera el embedding de la consulta utilizando un modelo transformer
    query_emb = sentence_transformer_model.encode([cln_query], 
                                                  device='cuda',  # Aceleración con GPU
                                                  normalize_embeddings=True)  # Normaliza los embeddings
    
    # Calcula la similitud del coseno entre la consulta y cada documento en el corpus (mediante embeddings)
    scores = (query_emb @ corpus_embeddings_matrix.T)[0]
    
    return scores

def hybrid_query_rrf(query: str, sintactic_retriever: partial, semantic_retriever: partial, k: int = 60) -> np.array:
    """
    Combina los puntajes de consultas sintácticas y semánticas utilizando la técnica de 
    Reciprocal Rank Fusion (RRF), que asigna puntajes inversos a las posiciones de los documentos
    en las listas de resultados.

    Args:
        query (str): Consulta de entrada.
        sintactic_retriever (partial): Función de consulta sintáctica (BM25 o TF-IDF).
        semantic_retriever (partial): Función de consulta semántica (embeddings).
        k (int, optional): Constante que controla la magnitud de los puntajes RRF. Default: 60.

    Returns:
        np.array: Puntajes finales fusionados de los documentos.
    """    
    
    # Recupera puntajes sintácticos y semánticos
    sintactic_scores = sintactic_retriever(query)
    semantic_scores = semantic_retriever(query)
    
    # Calcula los rangos inversos (mayor puntaje -> rango 1) para ambas consultas
    sintactic_ranks = sintactic_scores.argsort()[::-1].argsort() + 1  # Rango de 1 para el más relevante
    semantic_ranks = semantic_scores.argsort()[::-1].argsort() + 1
    
    # Fusiona los puntajes utilizando la fórmula RRF
    rrf_scores = (1 / (k + sintactic_ranks)) + (1 / (k + semantic_ranks))
    
    return rrf_scores

def hybrid_query_avg(query: str, sintactic_retriever: partial, semantic_retriever: partial, 
                     alpha: float = 0.5) -> np.array:
    """
    Combina los puntajes sintácticos y semánticos utilizando un promedio ponderado, donde el
    parámetro `alpha` controla el peso dado a cada tipo de consulta.

    Args:
        query (str): Consulta de entrada.
        sintactic_retriever (partial): Función de consulta sintáctica.
        semantic_retriever (partial): Función de consulta semántica.
        alpha (float, optional): Peso asignado a la consulta semántica en el promedio. Defaults to 0.5.

    Returns:
        np.array: Puntajes finales combinados.
    """      
    
    # Recupera puntajes sintácticos y normaliza entre 0 y 1
    sintactic_scores = sintactic_retriever(query)
    sintactic_scores = (sintactic_scores - sintactic_scores.min()) / (sintactic_scores.max() - sintactic_scores.min())
    
    # Recupera puntajes semánticos y normaliza entre 0 y 1
    semantic_scores = semantic_retriever(query)
    semantic_scores = (semantic_scores - semantic_scores.min()) / (semantic_scores.max() - semantic_scores.min())
    
    # Calcula el puntaje combinado usando un promedio ponderado
    scores = alpha * semantic_scores + (1 - alpha) * sintactic_scores

    return scores


## Carga de archivos (baseline)

In [4]:
# Mismo script que viene en el nb del baseline 

# Función para cargar los relevamientos (qrels) y relacionar pasajes con identificadores de documentos
def load_qrels(docs_dir: str, fqrels: str) -> Dict[str, Dict[str, int]]:
    """
    Carga los relevamientos de consultas y pasajes desde archivos JSON.

    Esta función asocia cada ID de pasaje a un documento y luego genera una estructura
    que permite vincular preguntas (QuestionID) con pasajes relevantes (PassageID).
    
    Args:
        docs_dir (str): Directorio donde se encuentran los documentos estructurados (en formato JSON).
        fqrels (str): Archivo JSON que contiene el mapeo de las preguntas y sus pasajes relevantes.

    Returns:
        Dict[str, Dict[str, int]]: Un diccionario con el siguiente formato:
            {
                "QuestionID": {
                    "PassageID": 1
                }
            }
            Donde 1 indica que el pasaje es relevante para esa pregunta.
    """    
    # Número de documentos a procesar (asumido en 40, puede ser parametrizado)
    ndocs = 40
    docs = []  # Lista para almacenar los documentos leídos

    # Lee cada uno de los archivos de documentos JSON
    for i in range(1, ndocs + 1):
        with open(os.path.join(docs_dir, f"{i}.json")) as f:
            doc = json.load(f)  # Carga el contenido JSON del archivo
            docs.append(doc)  # Añade el documento a la lista

    # Mapea DocumentID y PassageID a IDs de pasajes individuales
    did2pid2id: Dict[str, Dict[str, str]] = {}
    
    # Itera sobre cada documento y mapea los pasajes
    for doc in docs:
        for psg in doc:
            # Si el DocumentID no está en el diccionario, lo inicializa
            did2pid2id.setdefault(psg["DocumentID"], {})
            # Asegura que no haya duplicados de ID en el DocumentID
            assert psg["ID"] not in did2pid2id[psg["DocumentID"]]
            # Asocia el PassageID con el ID dentro del diccionario
            did2pid2id[psg["DocumentID"]].setdefault(psg["PassageID"], psg["ID"])

    # Carga el archivo de relevancia (qrels) para asociar preguntas con pasajes relevantes
    with open(fqrels) as f:
        data = json.load(f)
    
    qrels = {}  # Diccionario para almacenar los relevamientos (query relevance)
    
    # Recorre las preguntas y los pasajes asociados
    for e in data:
        qid = e["QuestionID"]  # Obtiene el QuestionID de la consulta
        for psg in e["Passages"]:  # Recorre los pasajes relevantes para esa pregunta
            qrels.setdefault(qid, {})  # Inicializa el diccionario para el QuestionID si no existe
            # Mapea el PassageID al ID del pasaje real, usando did2pid2id
            pid = did2pid2id[psg["DocumentID"]][psg["PassageID"]]
            # Marca el pasaje como relevante (1) para esa consulta
            qrels[qid][pid] = 1
    
    return qrels  # Devuelve la estructura de relevamientos


# Carga los relevamientos desde el directorio de documentos y el archivo de preguntas
file_type = 'test'
qrels = load_qrels("ObliQADataset/StructuredRegulatoryDocuments", f"ObliQADataset/ObliQA_{file_type}.json")

# Escribe los relevamientos en un archivo en formato de texto con cada línea representando
# un QuestionID, Q0, PassageID y relevancia (1) para el formato TREC
with open("qrels", "w") as f:
    for qid, rels in qrels.items():
        for pid, rel in rels.items():
            # Formatea cada línea en el formato estándar: QuestionID, Q0, PassageID, Relevance
            line = f"{qid} Q0 {pid} {rel}"
            f.write(line + "\n")  # Escribe la línea en el archivo de salida


# Proceso para cargar una colección de pasajes desde el directorio de documentos estructurados
ndocs = 40  # Número de documentos a procesar
collection = []  # Lista para almacenar la colección de pasajes

# Lee cada documento y extrae los pasajes relevantes
for i in range(1, ndocs + 1):
    with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)  # Carga el contenido del documento JSON
        for psg in doc:  # Recorre cada pasaje del documento
            # Añade cada pasaje a la colección como un diccionario con los campos relevantes si
            # la longitud del texto es mayor a 50 caracteres (para evitar pasajes vacíos o irrelevantes por
            # su corta longitud) 
            if len(psg["PassageID"] + " " + psg["Passage"])>100: # Mejora propuesta por el grupo
                collection.append(
                    dict(
                        text=psg["PassageID"] + " " + psg["Passage"],  # Combina el PassageID y el texto del pasaje
                        ID=psg["ID"],  # ID del pasaje
                        DocumentId=psg['DocumentID'],  # ID del documento
                        PassageId=psg['PassageID'],  # ID del pasaje
                    )
                )

### Sparse sintactic representation: BM25

In [5]:
# Tokenizamos y limpiamos el corpus (colección de documentos)
# Cada documento en 'collection' se limpia usando la función clean_text y luego se tokeniza.
# La función tokenizer genera unigramas y bigramas para cada documento.
tokenized_corpus = [tokenizer(clean_text(doc['text'])) for doc in collection]

# Inicializamos el modelo BM25 con el corpus tokenizado
# BM25 es un modelo de recuperación de información que pondera la relevancia de los documentos basándose
# en la frecuencia de términos, la longitud promedio de documentos y dos hiperparámetros k1 y b:
# - k1 controla qué tan fuertemente se pondera la frecuencia del término (TF).
# - b controla cuánto impacto tiene la longitud del documento.
bm25 = BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)

# Convertimos la colección original de documentos en un array de NumPy para facilitar su manejo posterior.
# Este array es más eficiente para operaciones que requieren acceso por índice o que involucran grandes conjuntos de datos.
collection_array = np.array(collection)

# Verificamos la longitud del corpus tokenizado (número de documentos procesados).
# Esto es útil para asegurarnos de que todos los documentos hayan sido correctamente tokenizados.
len(tokenized_corpus) # 10592 (originalmente 13732)

10592

### Sparse sintactic representation: TF-IDF

In [6]:
# Inicializamos un vectorizador TF-IDF con bigramas y un conjunto de parámetros para optimizar la representación.
# Parámetros:
# - ngram_range=(1, 2): Utiliza tanto unigramas como bigramas.
# - max_features=30000: Limita la matriz TF-IDF a las 30,000 características (términos) más frecuentes.
# - max_df=0.9: Ignora términos que aparezcan en más del 90% de los documentos (considerados como demasiado comunes).
# - min_df=3: Ignora términos que aparezcan en menos de 3 documentos (considerados como demasiado raros).
# - stop_words='english': Utiliza la lista de stopwords en inglés para eliminar términos irrelevantes.
# - preprocessor=clean_text: Aplica la función de limpieza del texto antes de vectorizar los documentos.
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=30000, 
                                   max_df=0.9, min_df=3, stop_words='english', 
                                   preprocessor=clean_text
                                   )

# Transformamos el corpus en una matriz TF-IDF, donde cada documento se representa como un vector de características
# basadas en las frecuencias ponderadas de términos (utilizando tanto unigramas como bigramas).
corpus_tfidf_matrix = tfidf_vectorizer.fit_transform([doc['text'] for doc in collection_array])

/home/raul/Escritorio/extra/misis/rl/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn(


### Dense semantic representation: sentence-transformers/all-MiniLM-L6-v2

In [7]:
# Codificamos los documentos en embeddings semánticos utilizando un modelo preentrenado basado en transformers.
# Parámetros:
# - device='cuda': Utiliza la GPU para acelerar el cálculo de embeddings.
# - normalize_embeddings=True: Normaliza los embeddings para asegurar que las magnitudes de los vectores no dominen
#   las comparaciones, lo cual es útil para cálculos de similitud del coseno.
# - show_progress_bar=True: Muestra una barra de progreso durante el proceso de codificación.
# - max_length=512: Limita la longitud máxima de los textos a 512 tokens (esto es útil para modelos de transformers,
#   que suelen tener límites de longitud en la entrada).
corpus_embeddings_matrix = sentence_transformer_model.encode([i['text'] for i in collection_array],
                          device='cuda',
                          normalize_embeddings=True,
                          show_progress_bar=True,
                          max_length=512,
                          )

Batches:   0%|          | 0/331 [00:00<?, ?it/s]

In [8]:
# Configuramos los recuperadores semántico y sintáctico

# Utilizamos `partial` para crear una función de recuperación semántica personalizada.
# El objetivo es crear una función que ya esté configurada con la matriz de embeddings precomputada y el modelo de embeddings.
# Esto permite pasar directamente la consulta sin tener que recalcular o volver a cargar estos recursos.
semantic_retriever = partial(semantic_query, corpus_embeddings_matrix=corpus_embeddings_matrix,
                             sentence_transformer_model=sentence_transformer_model)

# De manera similar, creamos una función de recuperación sintáctica basada en el modelo TF-IDF.
# Esta función toma la matriz de TF-IDF del corpus y el vectorizador TF-IDF, permitiendo ejecutar consultas 
# directamente sin necesidad de recalcular la matriz o el vectorizador.
sintactic_tfidf_retriever = partial(sintactic_query_tfidf, corpus_tfidf_matrix=corpus_tfidf_matrix,
                                   tfidf_vectorizer=tfidf_vectorizer)

# Creamos una función de recuperación sintáctica utilizando el modelo BM25.
# En este caso, el `bm5_instance` ya está configurado con el corpus tokenizado, permitiendo realizar consultas
# sobre el modelo BM25 directamente.
sintactic_bm25_retriever = partial(sintactic_query_bm5, bm5_instance=bm25)

In [17]:
# Casi el Mismo script que viene en el nb del baseline 

# Diccionario para almacenar los resultados recuperados para cada consulta (QuestionID)
retrieved = {}
# Número de documentos a recuperar por cada consulta
top_n = 10

# Abrimos el archivo JSON que contiene las consultas de prueba (ObliQA_test.json)
with open("ObliQADataset/ObliQA_test.json") as f:
    data = json.load(f)  # Cargamos el contenido del archivo JSON
    
    # Iteramos sobre cada entrada (pregunta) en el archivo de datos
    for e in tqdm(data):  # tqdm agrega una barra de progreso durante la iteración
        query = e['Question']  # Extraemos la pregunta o consulta desde el campo 'Question'
        
        # Realizamos una consulta híbrida combinando la recuperación sintáctica (BM25) y la semántica (embeddings)
        # El valor de alpha controla el peso que le damos a la consulta semántica vs. sintáctica. En este caso,
        # alpha=0.6 implica que el score semántico es ligeramente superior.
        scores = hybrid_query_avg(
                                query,
                                sintactic_retriever=sintactic_tfidf_retriever,  # Recuperador BM25 para consultas sintácticas
                                semantic_retriever=semantic_retriever,  # Recuperador de embeddings para consultas semánticas
                                alpha=0.6  # Peso ligeramente superior para la consulta semántica
                                )
        
        # Obtenemos los índices de los `top_n` documentos con mayor puntaje utilizando np.argpartition.
        # np.argpartition es más eficiente que ordenar completamente todos los documentos cuando solo necesitamos
        # los primeros `n` documentos.
        top_k = np.argpartition(-scores, top_n)[:top_n]
        
        # Ordenamos los índices de los `top_n` documentos de acuerdo con sus puntajes.
        # Esto asegura que los documentos se devuelvan en el orden correcto según sus puntuaciones.
        top_k = top_k[np.argsort(-scores[top_k])]

        # Recuperamos los documentos correspondientes a los índices seleccionados.
        # collection_array contiene el texto y los metadatos de todos los documentos, por lo que usamos top_k para
        # seleccionar solo los documentos más relevantes para la consulta.
        top_docs = collection_array[top_k]

        # También extraemos los puntajes correspondientes a estos documentos
        top_scores = scores[top_k]

        # Creamos una lista de diccionarios, donde cada diccionario contiene los detalles del documento
        # y su puntaje de relevancia.
        top_results = [{**doc, 'score': score} for doc, score in zip(top_docs, top_scores)]
        
        # Guardamos los resultados para la consulta actual en el diccionario 'retrieved'
        # donde la clave es el QuestionID y el valor es la lista de los documentos más relevantes.
        retrieved[e["QuestionID"]] = top_results

100%|██████████| 22295/22295 [05:01<00:00, 73.87it/s]


In [18]:
#pd.to_pickle(retrieved, "data/retrieved_train_hard_negatives.pkl")  

In [15]:
# Mismo script que viene en el nb del baseline 

# Escribimos los resultados recuperados en un archivo en formato TREC para evaluación
with open("rankings.trec", "w") as f:
    for qid, hits in retrieved.items():  # Iteramos sobre cada QuestionID y su lista de documentos relevantes
        for i, hit in enumerate(hits):  # Para cada documento recuperado, generamos una línea en formato TREC
            # Formato de línea: QuestionID, Q0, DocumentID, Rank, Score, Método
            line = f"{qid} 0 {hit['ID']} {i+1} {hit['score']} bm25"
            f.write(line + "\n")  # Escribimos la línea en el archivo

In [16]:
# Evaluación de la recuperación utilizando 'trec_eval'
# Utilizamos la herramienta 'trec_eval' para medir el rendimiento del sistema de recuperación en términos de
# métricas como MAP (Mean Average Precision) y Recall.
# Las opciones -m recall.10 y -m map_cut.10 indican que estamos evaluando el recall en los primeros 10 documentos
# y el MAP (precision promedio) en los primeros 10 documentos.

!trec_eval/trec_eval -m recall.10 -m map_cut.10 ./qrels ./rankings.trec

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


recall_10             	all	0.8062
map_cut_10            	all	0.6698
